In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()


In [3]:
from agents import Agent, Runner, TResponseInputItem

simple_agent = Agent(
    name="Bob",
    instructions="You are a friendly assistant. Reply concisely."
)

convo: list[TResponseInputItem] = [
    {"content": "Hey!", "role": "user"},
    {"content": "Hey! How are you? How can I help you today?", "role": "assistant"},
    {"content": "Im having a bad day", "role": "user"}
]
result = await Runner.run(simple_agent, convo)
for message in result.to_input_list():
    print(message)

{'content': 'Hey!', 'role': 'user'}
{'content': 'Hey! How are you? How can I help you today?', 'role': 'assistant'}
{'content': 'Im having a bad day', 'role': 'user'}
{'id': 'msg_68b744a6b3988194bf58d6e7fca4e2d405935b9ca5a950d4', 'content': [{'annotations': [], 'text': "I'm sorry to hear that. Want to share what's going on? Maybe I can help or just lend an ear.", 'type': 'output_text', 'logprobs': []}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}


In [4]:
convo: list[TResponseInputItem] = []
print("You are now chatting with agent Bob. Type 'exit' to end the conversation.")
while True:
    user_input = input("You: ")
    print("You: " + user_input)

    if user_input == "exit":
        print("Goodbye!")
        break

    convo.append({"content": user_input, "role": "user"})
    result = await Runner.run(simple_agent, convo)

    print(f"Agent: {result.final_output}")

    convo = result.to_input_list()

You are now chatting with agent Bob. Type 'exit' to end the conversation.
You: Heyo, what's up?
Agent: Hey there! Just here to help out. What's up with you?
You: What's the weather like?
Agent: I can't provide real-time updates. You might want to check a weather app or site for the latest info.
You: okay, bye
Agent: Bye! Take care!
You: exit
Goodbye!


In [5]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples"
)

def on_math_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to math tutor agent")

# This agent has the capability to handoff to either the history or math tutor agent
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question." +
    "If neither agent is relevant, provide a general response.",
    handoffs=[history_tutor_agent, handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)

convo: list[TResponseInputItem] = []
last_agent = triage_agent
print("You are now chatting with the triage agent. Type 'exit' to end the conversation.")
while True:
    user_input = input("You: ")
    print("You: " + user_input)

    if user_input == "exit":
        print("Goodbye!")
        break

    convo.append({"content": user_input, "role": "user"})
    result = await Runner.run(last_agent, convo)

    convo = result.to_input_list()
    last_agent = result.last_agent

    print(f"{last_agent.name}: {result.final_output}\n")

You are now chatting with the triage agent. Type 'exit' to end the conversation.
You: Uhm. What is this for?
Triage Agent: I'm here to help with your homework questions related to history or math. If you have a specific question in either of those subjects, feel free to ask!

You: Can you help me with algebra?
Handing off to math tutor agent
Math Tutor: Of course! What algebra question do you need help with?

You: How much do I need to bake cookies?
Math Tutor: For that, let me know how many cookies you want to bake and if you have a specific recipe. Also, how is this related to algebra? Are you trying to figure out ingredient amounts using equations?

You: I guess I'm more interested in the history, after all
Math Tutor: No problem! What historical topic are you curious about?

You: exit
Goodbye!
